# Feature Engineering 

This file is a list of functions that would generate the final data set based on EDA for models to learn.

- preparation
- categories and shops cleaning functions
- data grouping function
- encoding function
- grid generation function
- adding encodings and additional months, shops and items features
- target lagging function
- knn features Function
- encodings lagging functions
- index-based features function
- tfidf (pca) features function
- final function
- save generated data

### preparation

Connect to google drive, import packages, define helper function and data path variable

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
cd '/content/gdrive/My Drive/EY/submission'

/content/gdrive/My Drive/Final_Project/DP/submission


In [0]:
import numpy as np
import pandas as pd 
import os
import re
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from itertools import product
import pickle
from sklearn.neighbors import NearestNeighbors
import scipy.stats

In [0]:
def downcast_dtypes(df):
    '''
    helper function: downcast dtypes from 64 to 32 bit to save memory 
    '''
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"] 
    cat_cols =   [c for c in df if df[c].dtype.name == 'category'] 
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    if len(cat_cols)>0:
        df[cat_cols]   = df[cat_cols].astype(np.int32)
    return df

In [0]:
data_place=os.path.join(os.getcwd(),'input&output/')

### categories and shops cleaning functions



These functions take the groupped data and make following modifications:
- clean the shops by merging duplicates and deleting useless shops;

- shops_to_change= [[0,1,11],[57,58,10]],
shops_to_delete=[8,9,20,23,27,29,30,32,33,40,54];
- clean the categories with items that are not in test.

In [0]:
def shops_clean(data_file,shops_to_change=[[0,1,11],[57,58,10]],shops_to_delete=[8,9,20,23,29,30,32,33,40,54]):
    '''
    the input is the grouped or initial data_set and 2 lists - shops to merge and shops to delete;
    '''
    #first merge the duplicate shops by adding the sales of one to the other;
    for i in range(len(shops_to_change[0])):
        data_file.loc[data_file["shop_id"]==shops_to_change[0][i],'shop_id']=shops_to_change[1][i]
    #next delete the useless shops
    data_file=data_file[np.logical_not(data_file["shop_id"].isin(shops_to_delete))]
    #output is clean data_set
    return data_file

def cat_clean(data_place,data_file,cat_to_del=[1,  4,  8, 10, 13, 14, 17, 18, 32, 39, 46, 48, 50, 51, 52, 53, 59, 66, 68, 80, 81, 82]):
    '''
    data place is the path to find items file, data_file is train data, cat_to_del is the list of categories to delete
    '''
    #read categories
    items = pd.read_csv(os.path.join(data_place, 'items.csv'))
    #map categories to train data
    data_file['cat']=data_file['item_id'].map(pd.Series(items['item_category_id'].values,index=items['item_id'].values))
    #next delete the useless categories
    data_file=data_file[np.logical_not(data_file['cat'].isin(cat_to_del))]    
    return data_file

### data grouping function


This function takes original data set as input and make following modifications:
- merge by month, item and shop;
- target is clipped;
- price is clipped by 4 std of given month; 
- unique shops and items are saved in a way how they appear in train_data - needed for index variables.

In [0]:
def group_data(data_file, clip=True,save_str=True):
    '''
    this file groups data per month, item and shop;
    data_file is initial file;
    clip is variable if to clip prices and sales or not;
    save_str is if to save order of shops and items per month; 
    '''
    if save_str==True:
        cur_shops=[]
        cur_items=[]
        for block_num in range(0,data_file["date_block_num"].max()+1):
            cur_shops += [data_file[data_file['date_block_num']==block_num]['shop_id'].unique()]
            cur_items += [data_file[data_file['date_block_num']==block_num]['item_id'].unique()]
    #first set the columns to merge on;
    index_cols = ['date_block_num','shop_id','item_id']
    #make grouping;
    temp=data_file.groupby(index_cols,as_index=False)
    #merge the sales and rename them to target;
    data_group=temp['item_cnt_day'].sum()
    data_group.rename(columns = {'item_cnt_day':'target'}, inplace = True)
    #add the max of prices
    data_group["item_price"]=temp["item_price"].max()["item_price"]
    #next stage is clipping;
    if clip==True:
        #clipping target for outliers; 
        data_group['target']=data_group['target'].clip(0,20)
        data_group=data_group[data_group['target']!=0]
        #prices are more complicated: clip by 4 std of log distribution; 
        clipped_price=[]
        for i in range(0,data_group["date_block_num"].max()+1):
            temp=data_group[data_group["date_block_num"]==i]['item_price']
            temp2=np.exp(np.log(temp).mean()+4*np.log(temp).std())
            if i==0:
                clipped_price=data_group[data_group["date_block_num"]==i]['item_price'].clip(0,temp2)
            else:
                clipped_price=pd.concat([clipped_price,data_group[data_group["date_block_num"]==i]['item_price'].clip(0,temp2)],axis=0)
        data_group["item_price"]=clipped_price
        
    if save_str==True:
        return [data_group,[cur_shops,cur_items]]
    else:
        return data_group

### encoding function



This function takes the data file and save encodings of shops, month and items by target and by price.

Encoding strategy:
The variables to encode:
- date;
- shop_id;
- item_id;

There will be 4 types of encoding for 3 categories: 
- average price per unique item of last month;
- average sales per unique item of last month; 
- number of unique items in last month; 
- proportion of sales made below max price of given item last month; 

**NOTE**:
Test do not provide price, target and number of unique items (except for general number for a month) data; As a result, all encodings are lagged by 1 month (so here take price and sales of last month);

Sales are given per item as the number of unique items change; 

Number of unique items can be a proxy of size of shop and rank of this parameter can show if shop is big or small; 

Proportion of items sold below max price can indicate the discounts; use max due to the assumption that shops are seldom allowed to increase price but may be allowed to lower price with discounts. 

**NOTE 2**:

Use log of the encodings as sales and prices; prices seem to be distributed as log norma; sales seem to look more like Poisson distribution but here use log because of the advantages of log which can show the dynamics. 
   

**NOTE 3**:

Normalize the encodings by subtracting the general encoding (monthly encoding) from category specific encoding - that will make mean to be closer to zero; Also it will indicate the ranking of shop or item against the general mean; 

The monthly encodings will be normed by rolling mean of previous periods.


In [0]:
def prepare_encodings(data_file, norm=True):
    '''
    this function takes the price and sales data per shop, per item and per date for future encodings;
    input is grouped data with clipped or not prices and sales;
    output is 3 lists to be used later on; 
    '''
    ##########step 0.1 to generate the volume of sales
    #classical price index is to take volume of sales and divide by number of sales to get price per unit sold;    
    data_file['volume']=data_file["item_price"]*data_file["target"]

    #########step 0.5 to find the times when sales was at price below max
    #first find max price for each item
    temp=data_file.groupby(['item_id',"date_block_num"])['item_price'].max().rename('max_price').reset_index()
    #then add this variable to general dataset through mapping trick:
    #as there is only one combination of item and month, merge then in one index and map needed data with it
    #such way is muuuuuch faster then merge; 
    temp['temp']=temp['item_id']*100+temp["date_block_num"]
    data_file['temp']=data_file['item_id']*100+data_file["date_block_num"]
    data_file['max_price']=data_file['temp'].map(pd.Series(temp['max_price'].values,index=temp['temp'].values))
    #then add low price as proxy of discount
    data_file['low_price']=(data_file['item_price']-data_file['max_price'])<0
    data_file['low_price_sale']=data_file['target']*data_file['low_price']    
    
    ##############step 1 to calculate price index and log price change per month (month encoding with price) 
    #first group by month
    temp=data_file.groupby(["date_block_num"])
    temp2=temp["target"].sum()
    #then find price per item: divide volume by number of items sold, name it and reset index;
    temp_date_enc=np.log((temp['volume'].sum()/(temp2+0.001)).rename('lprice_index')).reset_index()  # lprice: log price
    #next need the number of unique items;
    temp_date_enc['uni_index']=temp['item_id'].apply(lambda x: len(x.unique())).values
    #next sales per item - total sales divided by number of unique items; 
    temp_date_enc['ltarget_index']=np.log(temp2.values/temp_date_enc['uni_index']+1)
    #proportion of sales with low price to total sales;
    temp_date_enc['low_pr_index']=(temp['low_price_sale'].sum().values)/temp2
    #now prepare some centrolized features by substractive from feature its cummulative mean from previous dates;
    temp_date_enc['lprice_index_norm']=temp_date_enc['lprice_index']-temp_date_enc['lprice_index'].cumsum()/temp_date_enc.date_block_num
    temp_date_enc['ltarget_index_norm']=temp_date_enc['ltarget_index']-temp_date_enc['ltarget_index'].cumsum()/temp_date_enc.date_block_num
    temp_date_enc['low_pr_index_norm']=temp_date_enc['low_pr_index']-temp_date_enc['low_pr_index'].cumsum()/temp_date_enc.date_block_num
    temp_date_enc['uni_index_norm']=np.log(temp_date_enc['uni_index'])-np.log(temp_date_enc['uni_index']).cumsum()/temp_date_enc.date_block_num
    #shift the months by 1 - make the lagging;
    temp_date_enc["date_block_num"]=temp_date_enc["date_block_num"]+1

    ##############step 2 to generate price level of each shop normalized by current price level;
    temp=data_file.groupby(["date_block_num","shop_id"])
    temp2=temp["target"].sum()
    #again just divide grouped by shops volume of sales by sales in kind to get price index of shop and log it;
    temp_shop_enc=np.log(temp['volume'].sum()/(temp2)).rename('lprice_shop').reset_index()
    #next the number of unique items per shop;
    temp_shop_enc['uni_shop']=temp['item_id'].apply(lambda x: len(x.unique())).values
    #now sales;
    temp_shop_enc['ltarget_shop']=np.log(temp2.values/temp_shop_enc['uni_shop']+1)
    #proportion of sales with low price;
    temp_shop_enc['low_pr_shop']=(temp['low_price_sale'].sum().values)/temp2.values    
    #shift the months by 1
    temp_shop_enc["date_block_num"]=temp_shop_enc["date_block_num"]+1
    #normalizing the encodings;
    if norm==True:
        temp_shop_enc['lprice_shop']=temp_shop_enc['lprice_shop']-temp_shop_enc['date_block_num'].map(pd.Series(temp_date_enc["lprice_index"].values,index=temp_date_enc["date_block_num"].values))
        temp_shop_enc['ltarget_shop']=temp_shop_enc['ltarget_shop']-temp_shop_enc['date_block_num'].map(pd.Series(temp_date_enc['ltarget_index'].values,index=temp_date_enc["date_block_num"].values))
        temp_shop_enc['low_pr_shop']=temp_shop_enc['low_pr_shop']-temp_shop_enc['date_block_num'].map(pd.Series(temp_date_enc['low_pr_index'].values,index=temp_date_enc["date_block_num"].values))
        temp_shop_enc['uni_shop']=temp_shop_enc['uni_shop']/(temp_shop_enc['date_block_num'].map(pd.Series(temp_date_enc['uni_index'].values,index=temp_date_enc["date_block_num"].values))+0.001)
    
    ##############step 3 do alike with items
    temp=data_file.groupby(["date_block_num","item_id"])
    temp2=temp["target"].sum()
    #as with shops
    temp_item_enc=np.log(temp['volume'].sum()/(temp2)).rename('lprice_item').reset_index()
    #obviously for items not need number of unique items;
    temp_item_enc['ltarget_item']=np.log(temp2+1).values
    #proportion of sales with low price;
    temp_item_enc['low_pr_item']=(temp['low_price_sale'].sum().values)/temp2.values   
    temp_item_enc["date_block_num"]=temp_item_enc["date_block_num"]+1
    if norm==True:
        temp_item_enc['lprice_item']=temp_item_enc['lprice_item']-temp_item_enc['date_block_num'].map(pd.Series(temp_date_enc["lprice_index"].values,index=temp_date_enc["date_block_num"].values))
        temp_item_enc['ltarget_item']=temp_item_enc['ltarget_item']-temp_item_enc['date_block_num'].map(pd.Series(temp_date_enc['ltarget_index'].values,index=temp_date_enc["date_block_num"].values))
        temp_item_enc['low_pr_item']=temp_item_enc['low_pr_item']-temp_item_enc['date_block_num'].map(pd.Series(temp_date_enc['low_pr_index'].values,index=temp_date_enc["date_block_num"].values))
    
    return (temp_date_enc,temp_shop_enc,temp_item_enc)

### grid generation function



This function takes the true distribution of shops and items presented at given month to create grid of all items and shops combinations, add sales if they happened or add 0; 

After this function stage the test data can be added so that future features are generated in the same loops.

In [0]:
def grid_gen(df,temp_order=None,save_str=True):
    '''
    input is the grouped data and saved order of unique shops and items; 
    '''
    #first the columns to merge on
    index_cols = ['shop_id', 'item_id', 'date_block_num']
    #then make a grid as combination of all active shops and active items of given month;
    grid = [] 
    for block_num in range(0,df["date_block_num"].max()+1):
        #in case care about order
        if save_str==True:
            cur_shops = temp_order[0][block_num]
            cur_items = temp_order[1][block_num]
        #in case not care
        else:
            cur_shops = df[df['date_block_num']==block_num]['shop_id'].unique()
            cur_items = df[df['date_block_num']==block_num]['item_id'].unique()
        grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
    #convert grid to DataFrame
    grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
    #finally make the mapping trick to pass the sales values to grid
    grid['temp']=grid['item_id']*10000+grid['shop_id']*100+grid['date_block_num']
    df['temp']=df['item_id']*10000+df['shop_id']*100+df['date_block_num']
    grid['target']=grid["temp"].map(pd.Series(df["target"].values,index=df["temp"].values)).fillna(0)
    #drop temp
    grid=grid.drop(columns=["temp"])
    return grid


### adding encodings and additional months, shops and items features



In this stage:
- add encodings and new features of month:
 - 3 features that describe the month: month, long month dummy and December dummy and unique items of current month;
- add encodings and new features of shops: 
 - extract city name and encode it alike shops; add shops encodings to grid; 
- add encodings and new features of items:
 - add categories of items and encode them;
 - add additional features for categories: major categories and several dummies;
 - add date of first sale and dummy is this is new item;
 - add item encodings.

In [0]:
def add_gen_feat(df,temp_date_enc,norm=True):
    '''
    input is grid, saved encodings and bool either to norm data; 
    '''
    df["lprice_index"]=df["date_block_num"].map(pd.Series(temp_date_enc["lprice_index"].values,index=temp_date_enc["date_block_num"].values))
    df["ltarget_index"]=df["date_block_num"].map(pd.Series(temp_date_enc["ltarget_index"].values,index=temp_date_enc["date_block_num"].values))
    df["uni_index"]=df["date_block_num"].map(pd.Series(temp_date_enc["uni_index"].values,index=temp_date_enc["date_block_num"].values))
    df["low_pr_index"]=df["date_block_num"].map(pd.Series(temp_date_enc["low_pr_index"].values,index=temp_date_enc["date_block_num"].values))
    #fill na with 0 as NAs should be only at first month that will anyway be cut by lagging; 
    df["lprice_index"].fillna(0,inplace=True)
    df["ltarget_index"].fillna(0,inplace=True)
    df["uni_index"].fillna(0,inplace=True)
    df["low_pr_index"].fillna(0,inplace=True)
    #add normalized variables - 3 more variables usefull for linear models;
    if norm==True:
        df["lprice_index_norm"]=df["date_block_num"].map(pd.Series(temp_date_enc["lprice_index_norm"].values,index=temp_date_enc["date_block_num"].values))
        df["ltarget_index_norm"]=df["date_block_num"].map(pd.Series(temp_date_enc["ltarget_index_norm"].values,index=temp_date_enc["date_block_num"].values))
        df["low_pr_index_norm"]=df["date_block_num"].map(pd.Series(temp_date_enc["low_pr_index_norm"].values,index=temp_date_enc["date_block_num"].values))
        df['uni_index_norm']=df["date_block_num"].map(pd.Series(temp_date_enc['uni_index_norm'].values,index=temp_date_enc["date_block_num"].values))
        df["lprice_index_norm"].fillna(0,inplace=True)
        df["ltarget_index_norm"].fillna(0,inplace=True)
        df["low_pr_index_norm"].fillna(0,inplace=True)
        df['uni_index_norm'].fillna(0,inplace=True)
    #here add categories of month;
    df["month_cat"]=df["date_block_num"]%12+1
    #then the dummy for long monthes - they have additional day;
    df["long_m_dummy"]=(df["month_cat"]==1)*1+(df["month_cat"]==3)*1+(df["month_cat"]==5)*1+(df["month_cat"]==7)*1+(df["month_cat"]==8)*1+(df["month_cat"]==10)*1+(df["month_cat"]==12)*1
    #and also dummy for December;
    df["dec_dummy"]=(df["month_cat"]==12)*1
    #finally check the number of unique items - that is number of unique items of current month - test data allows us; 
    #That differes it with 'Uni_index' which is lagged;
    temp=df.groupby('date_block_num')['item_id'].apply(lambda x: len(x.unique())).rename("uni_index_now")
    df["uni_index_now"]=df['date_block_num'].map(temp)
    return df
    

In [0]:
def add_shop_feat(df,data_place,temp_shop_enc,norm=True):
    '''
    df is the dataframe where to add encodings and new features; 
    data_place is the place where data is stored to find shops file;
    temp_shop_enc is initially saved shop encodings with prices and sales;
    '''
    #part 1 read the shops file and, extract the city;
    shops = pd.read_csv(os.path.join(data_place, 'shops-translated.csv'))
    #get the city names;
    shops['city']=shops.iloc[:,0].apply(lambda x: x.split(" ")).apply(lambda x: x[0])
    #convert city name to categorical feature index;
    shops['city_id_cat']=shops['city'].astype('category').cat.rename_categories(range(len(shops['city'].unique())))
    
    #part 2 encode the city and shop by price and sales; 
    #take the saved encodings and add city id;
    temp_shop_enc["city_id_cat"]=temp_shop_enc["shop_id"].map(pd.Series(shops["city_id_cat"].values,index=shops["shop_id"].values))
    #then take mean of shops' encodings as encodings of city;
    #it is not 100% correct but it is alike geometric mean 
    temp2=temp_shop_enc.groupby(["date_block_num","city_id_cat"])
    temp_shop_enc=pd.merge(temp_shop_enc, temp2["lprice_shop"].mean().rename('lprice_city').reset_index(), on=["date_block_num",'city_id_cat'], how='left')
    temp_shop_enc=pd.merge(temp_shop_enc, temp2["ltarget_shop"].mean().rename('ltarget_city').reset_index(), on=["date_block_num",'city_id_cat'], how='left')
    temp_shop_enc=temp_shop_enc[temp_shop_enc["date_block_num"]<=df["date_block_num"].max()]
    
    #part 3 merging - tricky part are new shops (not often but happens) - they will lack data; 
    #take average of country to fill na;
    #taking city mean is not useful as new shops are often in new city;
    #so add city_id_cat seperately to ensure new shops to have city;
    df["city_id_cat"]=df["shop_id"].map(pd.Series(shops["city_id_cat"].values,index=shops["shop_id"].values))
    #not need city_id for merging encodings as each city has only one city;
    df=pd.merge(df, temp_shop_enc.drop(columns=["city_id_cat"]), on=["date_block_num","shop_id"], how='left')
    #fillna for prices as average of country;
    #finally take the number of unique items equal to zero at fillna as it is more for new shops; 
    df['uni_shop'].fillna(0,inplace=True)
    df['low_pr_shop'].fillna(0,inplace=True) 
    if norm==True:
        #in case data is normalized then fillna for prices is just 0
        df['ltarget_shop'].fillna(0,inplace=True)    
        df['ltarget_city'].fillna(0,inplace=True)
    else:
        df['ltarget_shop'].fillna(df["ltarget_index"],inplace=True)    
        df['ltarget_city'].fillna(df["ltarget_index"],inplace=True)   
    if norm==True:
        #in case data is normalized then fillna for prices is just 0
        df['lprice_shop'].fillna(0,inplace=True)    
        df['lprice_city'].fillna(0,inplace=True)
    else:
        df['lprice_shop'].fillna(df["lprice_index"],inplace=True)    
        df['lprice_city'].fillna(df["lprice_index"],inplace=True)    
    return df

In [0]:
def add_categ_test(df,data_place,temp_item_enc,norm=True):
    '''
    add category features;
    inputs are the data_frame, the place to look for data, and saved informaiton about encodings;
    also have bool to either clean the test data;
    output is new df with new features;
    '''
    #part 1: generating features from item_categories;
    item_cats = pd.read_csv(os.path.join(data_place, 'item_categories-translated.csv'))
    #the major categories;
    item_cats['major_name']=item_cats.iloc[:,0].apply(lambda x: x.lower().split(" - ")).apply(lambda x: x[0])
    #that is to make id of the categories;
    item_cats['major_cat']=item_cats['major_name'].astype('category').cat.rename_categories(range(len(item_cats['major_name'].unique())))
    #also some dummies for for found minor categories in Data analysis ;
    item_cats['is_PS']=item_cats['item_category_name'].str.contains('(?i)ps')*1
    item_cats['is_XBOX']=item_cats['item_category_name'].str.contains('(?i)xbox')*1
    item_cats['is_1C']=item_cats['item_category_name'].str.contains('(?i)1c')*1
    item_cats['is_digital']=item_cats['item_category_name'].str.contains('(?i)digit')*1 | item_cats['item_category_name'].str.contains('(?i)num')*1
    
    #part 2 get the items file;
    items = downcast_dtypes(pd.read_csv(os.path.join(data_place, 'items-translated.csv')))
    #Next add all new features to items file;
    items=pd.merge(items, item_cats.drop(columns=["item_category_name",'major_name']), on=['item_category_id'], how='left')
    #continue to use cat for categorical; there is always a place for cat; 
    items.rename(columns = {'item_category_id':'cat_cat'}, inplace = True)
    
    #Part 3: add the encodings - all 4 to categories and only 2 to major categories;
    #first add category id to encoding file;
    temp_item_enc["cat_cat"]=temp_item_enc["item_id"].map(pd.Series(items["cat_cat"].values,index=items["item_id"].values))
    #then encode it alike with cities;
    temp=temp_item_enc.groupby(["date_block_num","cat_cat"])
    temp_item_enc=pd.merge(temp_item_enc, temp["lprice_item"].mean().rename('lprice_cat').reset_index(), on=["date_block_num",'cat_cat'], how='left')
    temp_item_enc=pd.merge(temp_item_enc, temp["ltarget_item"].mean().rename('ltarget_cat').reset_index(), on=["date_block_num",'cat_cat'], how='left')
    temp_item_enc=pd.merge(temp_item_enc, temp["low_pr_item"].mean().rename('low_pr_cat').reset_index(), on=["date_block_num",'cat_cat'], how='left')
    #categories will also have the unique values of items but lagged
    #current number of items in category may be added - but hardly it will work;
    temp_item_enc=pd.merge(temp_item_enc, temp["item_id"].apply(lambda x: len(x.unique())).rename('uni_cat').reset_index(), on=["date_block_num",'cat_cat'], how='left')
    
    #then add major id to encoding file;
    temp_item_enc['major_cat']=temp_item_enc["item_id"].map(pd.Series(items['major_cat'].values,index=items["item_id"].values))
    #then encode it alike with shops;
    temp=temp_item_enc.groupby(["date_block_num",'major_cat'])
    temp_item_enc=pd.merge(temp_item_enc, temp["lprice_item"].mean().rename('lprice_maj').reset_index(), on=["date_block_num",'major_cat'], how='left')
    temp_item_enc=pd.merge(temp_item_enc, temp["ltarget_item"].mean().rename('ltarget_maj').reset_index(), on=["date_block_num",'major_cat'], how='left')
    temp_item_enc=temp_item_enc[temp_item_enc["date_block_num"]<=df["date_block_num"].max()]
    
    #part 4 merging to df
    #first add the category and category variables;
    df=pd.merge(df,items.drop(columns=['item_name']), on=['item_id'], how='left')
    #then add all the rest of variables; 
    df=pd.merge(df,temp_item_enc.drop(columns=['cat_cat','major_cat']), on=["date_block_num",'item_id'], how='left')
    #finally make the cleaning
    #fillna for prices for average and target with 0
    if norm==True:
        #if norm then the mean price will be 0 as item price was centered;
        df['lprice_maj'].fillna(0,inplace=True)
        df['lprice_cat'].fillna(0,inplace=True)
    else:
        df['lprice_maj'].fillna(df["lprice_index"],inplace=True)
        df['lprice_cat'].fillna(df["lprice_index"],inplace=True)
    if norm==True:
        #if norm then the mean price will be 0 as item price was centered;
        df['ltarget_maj'].fillna(0,inplace=True)
        df['ltarget_cat'].fillna(0,inplace=True)
    else:
        df['ltarget_maj'].fillna(df["ltarget_index"],inplace=True)
        df['ltarget_cat'].fillna(df["ltarget_index"],inplace=True)
           
    df['low_pr_cat'].fillna(0,inplace=True)
    df['uni_cat'].fillna(0,inplace=True)   
    #for item use price of category
    df['lprice_item'].fillna(df['lprice_cat'],inplace=True)
    df['ltarget_item'].fillna(df['ltarget_cat'],inplace=True)
    df['low_pr_item'].fillna(0,inplace=True)  
    
    #then add the date of first sale
    df["item_start"]=df['item_id'].map(df.groupby('item_id')["date_block_num"].min())

    #normalization
    if norm==True:
        df['uni_cat']=df['uni_cat']/(df["uni_index"]+000000.1)  
        #for item start, delete current month and divide by 12 
        df["item_start"]=(df["item_start"]-df["date_block_num"])/12
        df['if_new']=(df["item_start"]==0)*1
    else:
        df['if_new']=(df["item_start"]==df["date_block_num"])*1

    return df

### target lagging function



Lag of target is one of the most powerful features.

**NOTE**: clip all the lagged targets so that they will show the model more correct history; but models seem to have problem with predicting 20 so add the bool if lagged target was clipped - it may help to enforce predicting 20.

In [0]:
def make_lag_target(df,lag_value_target,norm=True,clip=True,clip_max=20):
    '''
    df is the dataframe with values;
    lag_value_target is the list of the lags for target.
    '''
    #step 0 create temp index for each combination of month, shop and item as unique value;
    df['temp']=df['date_block_num']*10000000+df['shop_id']*100000+df['item_id']  
    #loop
    for month_shift in lag_value_target:
    #instead of merge use map for target - much faster;
    #make second index with shifted month;
        df['temp_2']=(df['date_block_num'] + month_shift)*10000000+df['shop_id']*100000+df['item_id'] 
    #do mapping of initial index by shifted data;
        df['{}_lag_{}'.format('target', month_shift)]=df['temp'].map(pd.Series(df['target'].values,index=df['temp_2'].values))
    #NA filling with zero;
        df['{}_lag_{}'.format('target', month_shift)].fillna(0,inplace=True)
        if clip==True:
            df['{}_{}'.format('if_cl', month_shift)]=(df['{}_lag_{}'.format('target', month_shift)]>clip_max)*1
            df['{}_lag_{}'.format('target', month_shift)]=df['{}_lag_{}'.format('target', month_shift)].clip(0,clip_max)
        if norm==True:
            df['{}_lag_{}'.format('target', month_shift)]=df['{}_lag_{}'.format('target', month_shift)]/20
    df=df.drop(columns=['temp','temp_2'])
    return df

### knn features Function



Ideas is:
- Train model on last month data to ensure no leakages;
- Use Knn to find the in last month the items/shops that have same lagged sales hoping that they will have alike sales in current period.

features are:
- most common target value of neighbours and how many neighbors have it;
- weighted mean value of target weighted by distance;
- minimal distance neighbor and its target;
- median distance neighbor and its target;
- maximum predicted target. 

In [0]:
def knn_feat(X,Y,X_pred,k_list=7):
    #model;
    NN = NearestNeighbors(n_neighbors=k_list, metric='minkowski', n_jobs=-1,  algorithm= 'auto')
    #fit model on X_lag;
    NN.fit(X)
    #making predictions;
    NN_output = NN.kneighbors(X_pred)
    #finding target of predictions;
    Y_pred=[]
    for i in range(NN_output[1].shape[0]):
        Y_pred+=[Y.iloc[NN_output[1][i]].values.reshape(-1).astype('int64')]
    Y_pred=np.vstack(Y_pred)
    #creating the predictions;
    result=[]
    #find the prediction by majority;
    temp=scipy.stats.mode(Y_pred,1)
    result+=[np.hstack(temp[0])/20]
    result+=[np.hstack(temp[1])/k_list]
    #weighted by distance prediction;
    result+=[np.sum(np.multiply(NN_output[0],Y_pred),axis=1)/20]
    #prediction by min distance - they are the first in distance;
    #predictions are already sorted so take just the first one;
    result+=[np.array([x[0] for x in NN_output[0]])]
    result+=[np.array([x[0] for x in Y_pred])/20] 
    #prediction by median distance - they are the first in distance;
    result+=[np.array([x[int(np.floor(k_list/2))] for x in NN_output[0]])]
    result+=[np.array([x[int(np.floor(k_list/2))] for x in Y_pred])/20] 
    #maximum predicted label;
    result+=[np.array([np.max(x) for x in Y_pred])/20] 
    #returning;
    result=np.vstack(result)
    return np.transpose(result)

In [0]:
def knn_feat_full(df,start,feat,k_list=7,norm=True,special=True,shw_time=True):
    '''
    here either take full data or the splitted one;
    df is dataframe, start is the month to start to generate the features.
    '''
    if special==True:
        temp=df[np.logical_or(df['target_lag_1']>0,df['if_new']==1)][feat].copy()
    else:
        temp=df[feat].copy()
    fin_res=[]
    #that is a basic loop for calculating the knn features
    #normalize by square root - that would help the distribution to be closer to normal
    #save all in feature fin_res
    for i in range(start,temp['date_block_num'].max()+1):
        if shw_time==True: z=time.time()
        X=np.sqrt(temp[temp['date_block_num'].isin([i-1])].drop(columns=['date_block_num','target'])+3/8)
        Y=temp[temp['date_block_num'].isin([i-1])]['target']
        X_pred=np.sqrt(temp[temp['date_block_num'].isin([i])].drop(columns=['date_block_num','target'])+3/8)
        fin_res+=[knn_feat(X,Y,X_pred,k_list=7)]
        if shw_time==True: print(str(i),' is done', "time taken", str(time.time()-z))
    temp=np.vstack(fin_res)
    #save data seperatly as they will be added to main DF later in code
    temp=pd.DataFrame(temp)
    temp.columns=['mode_knn','mode_dist_knn','mean_knn','min_knn','min_dist_knn','med_knn','med_dist_knn','max_knn']
    return temp

### encodings lagging functions



Lag shop and item encodings to save RAM which needs to do quite a lot of mappings.

Normalization is done only for monthly encoding by taking the difference between months - that is features to reflect dynamics.

In [0]:
def make_lag_gen(df,add_cols_lag,temp_date_enc,norm=True):
    '''
    this function lags the general features;
    '''
    temp=temp_date_enc[["lprice_index","ltarget_index","uni_index","low_pr_index"]].copy()
    if norm==True:
        #if normalization needed, just look at the difference between periods;
        #uni_index is not logged so do it now for diff to make sence
        temp.loc[:,"uni_index"]=np.log(temp.loc[:,"uni_index"])
        temp=temp.diff()
    for month_shift in add_cols_lag:
        #now making the monthly shift and mapping like with target; 
        temp["date_block_num"]=temp_date_enc["date_block_num"]+month_shift
        df['{}_lag_{}'.format("lprice_index", month_shift)]=df["date_block_num"].map(pd.Series(temp["lprice_index"].values,index=temp["date_block_num"].values)).fillna(0)
        df['{}_lag_{}'.format("ltarget_index", month_shift)]=df["date_block_num"].map(pd.Series(temp["ltarget_index"].values,index=temp["date_block_num"].values)).fillna(0)
        df['{}_lag_{}'.format("uni_index", month_shift)]=df["date_block_num"].map(pd.Series(temp["uni_index"].values,index=temp["date_block_num"].values)).fillna(0)
        df['{}_lag_{}'.format("low_pr_index", month_shift)]=df["date_block_num"].map(pd.Series(temp["low_pr_index"].values,index=temp["date_block_num"].values)).fillna(0)
    return df

def make_lag_shop(df,add_cols_lag,temp_shop_enc):
    '''
    this function lags for the shops
    '''
    temp=temp_shop_enc[['lprice_shop', 'uni_shop', 'ltarget_shop','low_pr_shop']].copy()
    #no need for normalization as it was already done at encoding level
    df['temp']=df['date_block_num']*100+df['shop_id']    
    for month_shift in add_cols_lag:
        #now making the monthly shift and mapping;
        temp['temp']=(temp_shop_enc["date_block_num"]+month_shift)*100+temp_shop_enc['shop_id']
        df['{}_lag_{}'.format('lprice_shop', month_shift)]=df["temp"].map(pd.Series(temp['lprice_shop'].values,index=temp["temp"].values)).fillna(0)
        df['{}_lag_{}'.format('uni_shop', month_shift)]=df["temp"].map(pd.Series(temp['uni_shop'].values,index=temp["temp"].values)).fillna(0)
        df['{}_lag_{}'.format('ltarget_shop', month_shift)]=df["temp"].map(pd.Series(temp['ltarget_shop'].values,index=temp["temp"].values)).fillna(0)
        df['{}_lag_{}'.format('low_pr_shop', month_shift)]=df["temp"].map(pd.Series(temp['low_pr_shop'].values,index=temp["temp"].values)).fillna(0)
        temp=temp.drop(columns=['temp'])
    return df

def make_lag_item(df,add_cols_lag,temp_item_enc):
    ''' 
    this function lags for the items
    '''
    temp=temp_item_enc[['lprice_item','ltarget_item','low_pr_item']].copy()
    #no need for normalization as it was already done at encoding level
    df['temp']=df['date_block_num']*100+df['item_id']    
    for month_shift in add_cols_lag:
        #now making the monthly shift and mapping
        temp['temp']=(temp_item_enc["date_block_num"]+month_shift)+temp_item_enc['item_id']*100
        df['{}_lag_{}'.format('lprice_item', month_shift)]=df["temp"].map(pd.Series(temp['lprice_item'].values,index=temp["temp"].values)).fillna(0)
        df['{}_lag_{}'.format('ltarget_item', month_shift)]=df["temp"].map(pd.Series(temp['ltarget_item'].values,index=temp["temp"].values)).fillna(0)
        df['{}_lag_{}'.format('low_pr_item', month_shift)]=df["temp"].map(pd.Series(temp['low_pr_item'].values,index=temp["temp"].values)).fillna(0)
        temp=temp.drop(columns=['temp'])
    return df


### index-based features function



In this part the rank features are added; Idea is that shops are given in near linear order and items are not sorted in test data. So, items are added by rolling of unique items per shop. 

First items for first shop will have high probability of sale due to the way items are stacked; Such trend will preserve for first shops and decrease for further shops - see EDA.

Features will be added:
- order of shop and item in reverse order (biggest number correspond to first) and normalize by number of shops/items that month;
- rolling means of target lag 1 for items for each shop; Rolling at 10 and 100 windows; Normalize by total mean.


In [0]:
def index_feat(df,temp_order):
    '''
    input df as data frame that have lagged target;
    temp_order is needed as it has unique values of shops and items - to save time;
    below are working variables;
    '''
    result_sh=[]
    index_sh=[]
    result_it=[]
    index_it=[]
    res_rol_10=[]
    res_rol_100=[]
    #in this part look at the index value of shops and items relative to the month;
    #make it reverse order and divide by number of unique shops/items - in this case first shop/item always have value 1;
    for i in range(df['date_block_num'].min(),df['date_block_num'].max()+1):
        if i<(len(temp_order[0])):
            #so this part is for shops - find all unique;
            temp=temp_order[0][i]
            #make the index variable;
            result_sh+=[np.array(list(range(1,len(temp)+1)[::-1]))/len(temp)]
            #make the index for mapping of new variable;
            index_sh+=[temp+100*i]
            #so this part is for items - find all unique;
            temp=temp_order[1][i]
            #make the index variable
            result_it+=[np.array(list(range(1,len(temp)+1)[::-1]))/len(temp)]
            #make the index for mapping of new variable;
            index_it+=[temp*100+i]
            #now in this part make 2 rolling variables;
            #for each shop in given month calculate rolling mean;
            #NA for first values fill with just values;
            for j in temp_order[0][i]:
                temp2=df[df['date_block_num']==i][df['shop_id']==j]['target_lag_1']
                temp3=temp2.rolling(10).mean().fillna(temp2).values
                res_rol_10+=[temp3-np.mean(temp3)]
                temp3=temp2.rolling(100).mean().fillna(temp2).values
                res_rol_100+=[temp3-np.mean(temp3)]
        else:
            #same but for month not in temp_order;
            temp=df[df['date_block_num']==i]['item_id'].unique()
            result_it+=[np.array(list(range(1,len(temp)+1)[::-1]))/len(temp)]
            index_it+=[temp*100+i]            
            temp=df[df['date_block_num']==i]['shop_id'].unique()
            result_sh+=[np.array(list(range(1,len(temp)+1)[::-1]))/len(temp)]
            index_sh+=[temp+100*i]
            for j in list(temp):
                temp2=df[df['date_block_num']==i][df['shop_id']==j]['target_lag_1']    
                temp3=temp2.rolling(10).mean().fillna(temp2).values
                res_rol_10+=[temp3-np.mean(temp3)]
                temp3=temp2.rolling(100).mean().fillna(temp2).values
                res_rol_100+=[temp3-np.mean(temp3)]           
       
    #finally mapping of new variables;
    df['temp']=df['shop_id']+100*df['date_block_num']        
    df['shop_index']=df['temp'].map(pd.Series(np.hstack(result_sh),index=np.hstack(index_sh)))        
    df['temp']=df['item_id']*100+df['date_block_num']        
    df['item_index']=df['temp'].map(pd.Series(np.hstack(result_it),index=np.hstack(index_it)))        
    #adding rolling;
    df['roll_10']=np.hstack(res_rol_10)
    df['roll_100']=np.hstack(res_rol_100)
    
    return df.drop(columns=["temp"])

### tfidf (pca) features function



Use Tfidf to extract valuable information from item names; extract most common words from test items names and then add as feature to all items' names; 

Tfidf is basic: clean data and then extract all words or combination that appear at least in 0.5% of test item names; 

As there is too many features, PCA will be used to shrink it to 30 features.

In [0]:
def vectorizer(df,data_place,min_df=0.005,n_components=30):    
    '''
    the idea is to take the words used in 0.5% names of test items as paramters- that can be useful
    '''
    #load the data;
    items = downcast_dtypes(pd.read_csv(os.path.join(data_place, 'items-translated.csv')))
    df_test = downcast_dtypes(pd.read_csv(os.path.join(data_place, 'test.csv.gz')))
    #take the names of the items from test;
    items_test=df_test["item_id"]
    items=pd.Series(items["item_name"],index=items["item_id"])
    #clean the names; 
    items_clean=items.str.replace(r'[^\w\s]', ' ').str.replace(r' +', ' ').apply(lambda x: x.lower().strip())
    #create the vectorizer
    vect = TfidfVectorizer(min_df=min_df, ngram_range=(1,2),stop_words='english').fit(items_test.map(items_clean).unique())
    #now vectorize the items in item_id in all data train and test merged;  
    train_items=df['item_id'].unique()
    temp=vect.transform(pd.Series(train_items,index=train_items).map(items_clean)).todense()
    #get some sparse matrix of one-hot for each word from Vectorizer
    #hower it is very sparse and some words are indeed a combination of other like PC and PC Gaming
    #use PCA to reduce dimension but more important to seperate these features as far as possible
    pca = PCA(n_components = n_components).fit(temp)
    temp2=pca.transform(temp)
    print("PCA explained ",int(pca.explained_variance_ratio_.cumsum()[-1]*100),"%")
    temp3=pd.DataFrame(temp2,index=train_items).reset_index()
    temp3.columns=["item_id"]+["pca_"+str(x) for x in range(n_components)]
    df=pd.merge(df,temp3,on='item_id', how='left')
    return df

### final function 



The important part is that return can be 2 dataframes - as mentioned in EDA it can be a good idea to split the data by target_lag_1 and if_new they are powerful predicting variables.


In [0]:
def gen_data(data_place,clip=True,clip_max=20,lags=[1,2,3,6,9,12],cols_lag=[1,2,3,6],split=True,min_df=0.005,n_components=30,norm=True):
    '''
    the input are: data_place is the path of data files; clip is whether clip the target to 20;
    lags is how many lags to take for target;
    cols_lag= is the lags of the encodings, split is whether split the data in 2 (based on lag_1), 
    min_df and n_components are for shrinking the TFDF of names;
    norm is to normalize (bring center closer to zero) or not - need for linear and not harmful for trees; 
    '''
    st = time.time()
    temp_time=time.time()
    #load data
    print("load_data")
    df = downcast_dtypes(pd.read_csv(os.path.join(data_place, 'sales_train.csv.gz')))
    print("time used: ",time.time()-temp_time)
    #first make cleaning of shops
    print('cat and shops clean')
    df=shops_clean(df)
    df=cat_clean(data_place,df)   
    #now grouping
    print('group data')
    df,temp_order=group_data(df, clip=clip,save_str=True)
    print("time used: ",time.time()-temp_time)
    #save encodings
    print('encodings saving')
    temp_date_enc,temp_shop_enc,temp_item_enc=prepare_encodings(df,norm=norm)
    print("time used: ",time.time()-temp_time)
    #create grid for trainng
    print("get_grid")
    df=grid_gen(df,temp_order,save_str=True)
    df = downcast_dtypes(df)
    print("time used: ",time.time()-temp_time)
    
    #now unit the test data with train data so that all features are done in uniform way
    #add ID
    print("add_test")
    df['ID']=-1
    #read test
    df_test = downcast_dtypes(pd.read_csv(os.path.join(data_place, 'test.csv.gz')))
    #add date as 34 and target of 0
    df_test['date_block_num']=34
    df_test['target']=0
    #concat the data
    df=pd.concat([df,df_test],sort=False)
    print("time used: ",time.time()-temp_time)
    
    #now it is time to add encodings
    print("make encoding") 
    df=add_gen_feat(df,temp_date_enc,norm=norm)
    df=add_shop_feat(df,data_place,temp_shop_enc,norm=norm)
    df=add_categ_test(df,data_place,temp_item_enc,norm=norm)
    df = downcast_dtypes(df)
    print("time used: ",time.time()-temp_time)
    
    #now make lags
    print("make target lags") 
    df=make_lag_target(df,lag_value_target=lags,clip=clip,clip_max=clip_max)
    #here clip the target before knn
    df['target']=df['target'].clip(0,clip_max)
    print("time used: ",time.time()-temp_time)
    
    #next make knn features based n all target lags
    print("make knn encodings") 
    feat=['date_block_num','target']+list(df.columns[df.columns.str.contains('target_lag')])
    start=np.max(lags)
    k_list=7
    temp_knn=knn_feat_full(df,start,feat,special=True,shw_time=True)
    print("time used: ",time.time()-temp_time)
    
    #then cut the df by start month
    print("make encodings lags") 
    #cut the months that are not used and drop the index
    df=df[df['date_block_num']>=start]
    df=df.reset_index(drop=True)
    
    #now lags of encodings
    df=make_lag_gen(df,cols_lag,temp_date_enc,norm=norm)
    df=make_lag_shop(df,cols_lag,temp_shop_enc)
    df=make_lag_item(df,cols_lag,temp_item_enc)
    df=df.drop(columns=["temp"])
    print("time used: ",time.time()-temp_time)
    
    #making index features
    print("make index feature") 
    df=index_feat(df,temp_order)
    df = downcast_dtypes(df)
    print("time used: ",time.time()-temp_time)
    
    #make PCA vectors
    print("make vectors") 
    df=vectorizer(df,data_place,min_df=min_df,n_components=n_components)
    print("time used: ",time.time()-temp_time)
    
    #check that everything is downcasted
    print('last modifications')
    df = downcast_dtypes(df)
    print('TOTAL time used: ', (time.time()-st)/60, 'minutes')
    if split==True:
        temp=np.logical_or(df['target_lag_1']>0,df['if_new']==1)
        df_0=df[np.logical_not(temp)].reset_index(drop=True)
        df_1=df[temp].reset_index(drop=True)
        print("time used: ",time.time()-temp_time)
        return (df_0,pd.concat([df_1,temp_knn],axis=1))
    else:
        print("time used: ",time.time()-temp_time)
    
        return pd.concat([df,temp_knn],axis=1)

In [0]:
df_0,df_1=gen_data(data_place)

load_data
time used:  3.099313974380493
cat and shops clean
group data
time used:  6.660418510437012
encodings saving
time used:  7.76926064491272
get_grid
time used:  14.767264604568481
add_test
time used:  15.547861337661743
make encoding
time used:  40.45273685455322
make target lags
time used:  62.95737147331238
make knn encodings
12  is done time taken 24.813353061676025
13  is done time taken 18.374409198760986
14  is done time taken 16.336776733398438
15  is done time taken 16.587640047073364
16  is done time taken 15.676235437393188
17  is done time taken 13.800678253173828
18  is done time taken 14.87444519996643
19  is done time taken 15.516558170318604
20  is done time taken 14.539301633834839
21  is done time taken 16.524043798446655
22  is done time taken 19.03078866004944
23  is done time taken 22.185057401657104
24  is done time taken 22.181811809539795
25  is done time taken 13.971884727478027
26  is done time taken 13.244656085968018
27  is done time taken 13.615544080

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


time used:  512.6872396469116
make vectors
PCA explained  61 %
time used:  528.8023335933685
last modifications
TOTAL time used:  9.004768502712249 minutes
time used:  544.5114378929138


load_data
cat and shops clean
group data
encodings saving
get_grid
add_test
make encoding
time used:  40.45273685455322
make target lags
time used:  62.95737147331238
make knn encodings
12  is done time taken 24.813353061676025
13  is done time taken 18.374409198760986
14  is done time taken 16.336776733398438
15  is done time taken 16.587640047073364
16  is done time taken 15.676235437393188
17  is done time taken 13.800678253173828
18  is done time taken 14.87444519996643
19  is done time taken 15.516558170318604
20  is done time taken 14.539301633834839
21  is done time taken 16.524043798446655
22  is done time taken 19.03078866004944
23  is done time taken 22.185057401657104
24  is done time taken 22.181811809539795
25  is done time taken 13.971884727478027
26  is done time taken 13.244656085968018
27  is done time taken 13.615544080734253
28  is done time taken 10.355935335159302
29  is done time taken 10.007462739944458
30  is done time taken 10.601794242858887
31  is done time taken 10.545668363571167
32  is done time taken 12.716874361038208
33  is done time taken 13.410630464553833
34  is done time taken 14.05686354637146
time used:  416.70526242256165
make encodings lags
time used:  425.7609565258026
make index feature
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
time used:  512.6872396469116
make vectors
PCA explained  61 %
time used:  528.8023335933685
last modifications
TOTAL time used:  9.004768502712249 minutes
time used:  544.5114378929138

In [0]:
print('shape of df_1',df_1.shape)
print('shape of df_0',df_0.shape)

shape of df_1 (1165858, 139)
shape of df_0 (4809172, 131)


## save generated data 



### save part 1:
Start with saving the data with lag_1>0 and if_new; as it is compressed saving then columns are to be saved separately.

In [0]:
def save_sparce_short(df,data_place,name):
    '''
    save columns and values separatly
    '''
    temp_col=df.columns.values
    #columns to be saved in pickle
    with open(os.path.join(data_place,'generated_data/', str(name+'_cols.pickle')), 'wb') as file:
        pickle.dump(temp_col, file)    
    #saving the file
    name_train=name+'.gz'
    df.to_csv(os.path.join(data_place,'generated_data/', name_train), compression='gzip',index=False,header=False)

In [0]:
save_sparce_short(df_1,data_place,'data1')

### save part 2



The data with lag_1==0 is much larger and have 5 mln data records. This part is very slow and RAM intensive.

In [0]:
def save_sparce_long(df,data_place,name):
    #save columns
    temp_col=df.columns.values
    #columns to be saved in pickle
    with open(os.path.join(data_place,'generated_data/', str(name+'_cols.pickle')), 'wb') as file:
        pickle.dump(temp_col, file)
    #split the data on train and rest
    #save train data
    name_train=name+'_train.gz'
    #now save compressed file 
    df[df['date_block_num']<30].to_csv(os.path.join(data_place,'generated_data/', name_train), compression='gzip',index=False,header=False)
    #save second part
    name_train=name+'_ens.gz'
    df[df['date_block_num']>29].to_csv(os.path.join(data_place,'generated_data/', name_train), compression='gzip',index=False,header=False)

In [0]:
save_sparce_long(df_0,data_place,'data0')